In [ ]:
%load_ext autoreload
%autoreload 2

Data

https://www.abs.gov.au/statistics/labour/earnings-and-work-hours/weekly-payroll-jobs-and-wages-australia/week-ending-27-march-2021#data-download

https://www.abs.gov.au/statistics/labour/earnings-and-work-hours/weekly-payroll-jobs-and-wages-australia/week-ending-27-march-2021/6160055001_DO005.xlsx


In [ ]:
import pandas as pd

# note that this requires the openpyxl package to be installed
df = pd.read_excel("data/6160055001_DO005.xlsx", sheet_name=1)
df.shape

In [ ]:
df.head()

In [ ]:
import dtale

dtale.show(df)

In [ ]:
jobs_raw_df = pd.read_excel(
    "data/6160055001_DO005.xlsx",
    sheet_name=1,  # zero-indexed, so this is the second sheet!
    usecols="A:BO",
    skiprows=5,
    skipfooter=2,
)

jobs_raw_df.head()

In [ ]:
date_cols_df = jobs_raw_df.iloc[:, 2:]

ste_cols_df = (
    jobs_raw_df[jobs_raw_df.columns[0]]
    .str.split(r"\. ", expand=True)
    .rename(columns={0: "STE_CODE16", 1: "STE_NAME16"})
)

sa4_cols_df = (
    jobs_raw_df[jobs_raw_df.columns[1]]
    .str.split(r"\. ", expand=True)
    .rename(columns={0: "SA4_CODE16", 1: "SA4_NAME16"})
)

jobs_df = pd.concat([ste_cols_df, sa4_cols_df, date_cols_df], axis=1).melt(
    id_vars=["STE_CODE16", "STE_NAME16", "SA4_CODE16", "SA4_NAME16"],
    var_name="Date",
    value_name="Index",
)

jobs_df

In [ ]:
jobs_df.dtypes

In [ ]:
states_jobs = jobs_df.groupby(["STE_NAME16", "Date"])["Index"].mean()
states_jobs

In [ ]:
country_jobs = states_jobs.mean(level="Date")
country_jobs

## Visualising with Pandas/Matplotlib

In [ ]:
# convert to DataFrame for easier plotting

country_jobs_df = country_jobs.reset_index()
country_jobs_df

### Use Pandas' API to make Matplotlib plot

In [ ]:
%matplotlib inline

country_jobs_df.plot(
    x="Date", y="Index", title="Weekly Payroll Jobs and Wages Index"
);

### Improving Aesthetics of Matplotlib plots

**Option 1: use a theme**

In [ ]:
import matplotlib.pyplot as plt

plt.style.available

In [ ]:
plt.style.use("ggplot")

country_jobs_df.plot(
    x="Date", y="Index", title="Weekly Payroll Jobs and Wages Index"
);

**Technique 2: Increase the DPI**


In [ ]:
fig = plt.figure(dpi=300, figsize=(15, 5))

country_jobs_df.plot(
    x="Date", y="Index", ax=plt.gca(), title="Weekly Payroll Jobs and Wages Index"
);

### Use ipympl Widget for interactivity

_Note: this requires the ipympl package to be installed_

Let's plot the Index of all states. 

In [ ]:
%matplotlib widget

# to easily plot this with Pandas, first need to get it into wide format

states_jobs.unstack(level=0).plot(
    figsize=(15, 5), title="Weekly Payroll Jobs and Wages Index by State"
);

What if our data was in long (tidy) format?

In [ ]:
states_jobs_df = states_jobs.reset_index()
states_jobs_df

## Other plotting Options

**Seaborn**

In [ ]:
%matplotlib widget
import seaborn as sns

plt.figure(figsize=(13, 6))

sns.lineplot(
    data=states_jobs_df,
    x="Date",
    y="Index",
    hue="STE_NAME16",
).set(title="Weekly Payroll Jobs and Wages Index by State");

**Plotly**

In [ ]:
import plotly.express as px

px.line(
    states_jobs_df,
    x="Date",
    y="Index",
    color="STE_NAME16",
    title="Weekly Payroll Jobs and Wages Index by State",
    width=1200,
    height=500
)

Also consider **Holoviews** and **Altair**

### Adding Country-level mean

In [ ]:
states_and_country_df = pd.concat(
    [states_jobs_df, country_jobs_df.assign(STE_NAME16="AUS")]
)
state_names = list(states_jobs_df["STE_NAME16"].unique())

px.line(
    states_and_country_df,
    x="Date",
    y="Index",
    color="STE_NAME16",
    title="Weekly Payroll Jobs and Wages Index by State",
    color_discrete_map={"AUS": "black"},
    category_orders={"STE_NAME16": ["AUS"] + state_names},
    line_dash="STE_NAME16",
    line_dash_sequence=["dot"] + ["solid" for _state in state_names],
    width=1200,
    height=500
)

## Spatial Visualisation

In [ ]:
import geopandas as gpd

gdf = gpd.read_file("data/sa4_2016_aust_shape/SA4_2016_AUST.shp")

gdf

In [ ]:
import contextily as cx


def plot_wage_chloropleth(sa4_gdf, jobs_df, date):
    """Plot a chloropleth map of jobs Index for a given date"""
    # filter index data to current month and then join with geo data
    sa4_gdf = sa4_gdf[~sa4_gdf["geometry"].isnull()]
    filtered_df = jobs_df[jobs_df["Date"] == date][["SA4_CODE16", "Index"]]
    sa4_gdf = sa4_gdf.merge(filtered_df, on="SA4_CODE16", validate="one_to_one")

    fig, ax = plt.subplots()
    sa4_gdf.plot(
        ax=ax,
        edgecolor="black",
        column="Index",
        vmin=jobs_df["Index"].min(),
        vmax=jobs_df["Index"].max(),
    ).set(title="Australian Jobs and Wages Index")

    cx.add_basemap(ax, crs=gdf.crs.to_string(), source=cx.providers.CartoDB.Voyager)
    ax.axis("off")


plot_wage_chloropleth(gdf, jobs_df, "2020-01-04")

In [ ]:
import folium

# to get working properly
# use folium.GeoJson class with highlight_function callback that accesses the colormap
# import branca.colormap as cm
# colormap = cm.linear.YlGnBu_09.to_step(n=6, data=jobs_df["Index"])

def plot_wage_chloropleth_folium(sa4_gdf, jobs_df, date):
    """Plot a chloropleth map of jobs Index for a given date"""
    sa4_gdf = sa4_gdf[~sa4_gdf["geometry"].isnull()]
    filtered_df = jobs_df[jobs_df["Date"] == date][["SA4_NAME16", "Index"]]

    folium_map = folium.Map(location=[-22, 133], zoom_start=5)
    folium.LayerControl().add_to(folium_map)
    
    choropleth = folium.Choropleth(
        geo_data=sa4_gdf,
        data=filtered_df,
        columns=["SA4_NAME16", "Index"],
        key_on="feature.properties.SA4_NAME16",
        fill_color="YlGn",
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name="Wage Index",
        highlight=True,
    )
    choropleth.geojson.add_child(folium.features.GeoJsonTooltip(['SA4_NAME16'], labels=False))
    choropleth.add_to(folium_map)
    return folium_map


plot_wage_chloropleth_folium(gdf, jobs_df, "2020-01-04")